# Exemplo de Classificação Binária com RNA

Este notebook demonstra o uso da implementação de Rede Neural Artificial (RNA) para um problema de classificação binária.


## 1. Importação das Bibliotecas


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import sys
import os

# Configurações de visualização
plt.style.use('ggplot')
%matplotlib inline

## 2. Importação da RNA

Para usar este notebook no Google Colab, você precisará carregar a implementação da RNA.


In [2]:
import os
try:
    import google.colab
    IN_COLAB = True
    !git clone https://github.com/pedromandelli/RNA-Implementation.git
    %cd RNA-Implementation
    sys.path.insert(0, os.path.abspath(os.getcwd()))
except:
    IN_COLAB = False
    # Adicionar o diretório src ao PYTHONPATH se estiver executando localmente
    module_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
    if module_path not in sys.path:
        sys.path.insert(0, module_path)


In [3]:
# Importar a implementação da RNA
from src.rna import NeuralNetwork
print("Importação da NeuralNetwork bem-sucedida.")


Importação da NeuralNetwork bem-sucedida.


## 3. Carregamento e Exploração dos Dados

Nesta seção, carregaremos um conjunto de dados para o problema de classificação binária e faremos uma exploração inicial. Usaremos o dataset de Doença Cardíaca de Cleveland, que é um problema clássico de classificação binária na área da saúde


In [4]:
colunas = [
    "age", "sex", "cp", "trestbps", "chol", "fbs",
    "restecg", "thalach", "exang", "oldpeak", "slope",
    "ca", "thal", "target"
]

df = pd.read_csv('src/datasets/processed.cleveland.data', names=colunas, na_values="?")
df.dropna(inplace=True)
df["target"] = df["target"].apply(lambda x: 1 if x > 0 else 0)

# Visualizar as primeiras linhas do conjunto de dados

print("Primeiras 5 linhas do conjunto de dados:")
display(df.head())

FileNotFoundError: [Errno 2] No such file or directory: 'src/datasets/processed.cleveland.data'

In [ ]:
# Informações sobre o dataset
print(f"Formato do conjunto de dados: {df.shape}")
print(f"Número de classes: {df['target'].nunique()}")
print("Distribuição das classes:")
print(df['target'].value_counts())

In [ ]:
# Estatísticas descritivas
print("Estatísticas descritivas do conjunto de dados:")
display(df.describe())


## 4. Pré-processamento dos Dados

Nesta seção, realizaremos o pré-processamento necessário para preparar os dados para o treinamento da RNA.


In [ ]:
X = df.drop("target", axis=1)
y = df["target"].values.reshape(-1, 1)

colunas_numericas = ["age", "trestbps", "chol", "thalach", "oldpeak"]
colunas_categoricas = ["sex", "cp", "fbs", "restecg", "exang", "slope", "ca", "thal"]

# Normalizar os dados
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), colunas_numericas),
        ("cat", OneHotEncoder(), colunas_categoricas)
    ]
)

# Aplicar o pré-processamento em todos os dados
X = preprocessor.fit_transform(X)

# Dividir os dados em treino (60%), validação (20%) e teste (20%) - seguindo a orientação do professor

# Primeiro, separar 20% para teste
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y)

# Depois, separar 25% dos 80% restantes para validação (0.25 x 0.8 = 0.20)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp)

print(f"Tamanho do conjunto de treinamento: {X_train.shape}")
print(f"Tamanho do conjunto de validação: {X_val.shape}")
print(f"Tamanho do conjunto de teste: {X_test.shape}")
print(f"Distribuição das classes em treinamento:\n{pd.Series(y_train.flatten()).value_counts()}")
print(f"Distribuição das classes em validação:\n{pd.Series(y_val.flatten()).value_counts()}")
print(f"Distribuição das classes em teste:\n{pd.Series(y_test.flatten()).value_counts()}")


## 5. Visualização dos Dados

Vamos visualizar algumas características dos dados para entender melhor o problema.


In [ ]:
# Visualizar a distribuição das classes
plt.figure(figsize=(8, 6))
sns.countplot(x='target', data=df)
plt.title('Distribuição das Classes')
plt.xlabel('Classe (0: Sem doença, 1: Com doença)')
plt.ylabel('Contagem')
plt.show()

In [ ]:
# Selecionar algumas features numéricas para visualização
selected_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

# Criar um DataFrame só com as features selecionadas + target
df_selected = df[selected_features + ['target']]

# Transformar para formato long
df_selected_melted = df_selected.melt(id_vars='target', var_name='Feature', value_name='Valor')

# Usar displot para criar vários histogramas separados por feature
g = sns.displot(
    data=df_selected_melted,
    x='Valor',
    hue='target',
    col='Feature',
    col_wrap=3,
    kind='hist',
    bins=20,
    height=4,
    multiple='stack'
)

g.set_titles("{col_name}")
g.fig.suptitle("Distribuição das Features Numéricas por Classe", y=1.05)
plt.show()


In [ ]:
# Correlação
plt.figure(figsize=(14, 10))
correlation_matrix = df.corr()
mask = np.triu(correlation_matrix)
sns.heatmap(correlation_matrix, annot=True, mask=mask, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Matriz de Correlação entre Atributos')
plt.show()

## 6. Criação e Treinamento do Modelo

Nesta seção, criaremos e treinaremos o modelo de RNA para o problema de classificação binária.


In [ ]:
# Criar o modelo
model = NeuralNetwork(
    layer_sizes=[X_train.shape[1], 12, 6, 1],
    activation_functions=['relu', 'relu', 'sigmoid']
)

print("Arquitetura do modelo:")
print(model)


In [ ]:
# Treinar o modelo
history = model.fit(
    X_train, y_train,
    epochs=500,
    batch_size=32,
    learning_rate=0.003,
    loss_function='binary_crossentropy',
    optimizer='momentum',
    verbose=True,
    validation_data=(X_val, y_val)
)

## 7. Avaliação do Modelo

In [ ]:
# Fazer predições no conjunto de teste
y_pred_proba = model.predict(X_test, output_activation='sigmoid')

# Converter as probabilidades em classes (limiar 0.5)
y_pred_classes = (y_pred_proba > 0.5).astype(int).flatten()
y_test_classes = y_test.flatten()

# Calcular a acurácia
accuracy = accuracy_score(y_test_classes, y_pred_classes)
precision = precision_score(y_test_classes, y_pred_classes)
recall = recall_score(y_test_classes, y_pred_classes)
f1 = f1_score(y_test_classes, y_pred_classes)
roc_auc = roc_auc_score(y_test_classes, y_pred_proba)

print(f"Acurácia no conjunto de teste: {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"Precisão: {precision:.4f}")
print(f"Revocação (Recall): {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")

# Verificar se atende ao requisito do enunciado (erro < 50% -> acurácia > 50%)
print(f"O modelo {'atende' if accuracy > 0.5 else 'não atende'} ao requisito de desempenho.")

## 8. Visualização dos Resultados


In [ ]:

# Curva de aprendizado
plt.figure(figsize=(10, 6))
plt.plot(history['loss'], label='Training Loss')
if 'val_loss' in history:
    plt.plot(history['val_loss'], label='Validation Loss')
plt.title('Curva de Aprendizado')
plt.xlabel('Época')
plt.ylabel('Loss (Binary Cross-Entropy)')
plt.legend()
plt.show()


In [ ]:

# Visualizar a Matriz de Confusão (gráfico)
cm = confusion_matrix(y_test_classes, y_pred_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Sem doença', 'Com doença'],
            yticklabels=['Sem doença', 'Com doença'])
plt.title('Matriz de Confusão')
plt.xlabel('Classe Predita')
plt.ylabel('Classe Real')
plt.show()

## 9. Conclusão

Neste notebook, demonstramos o uso da nossa implementação de RNA para um problema de classificação binária. Utilizando o dataset Heart Disease (Cleveland), o modelo foi capaz de aprender a identificar a presença ou ausência de doença cardíaca com desempenho satisfatório, superando a acurácia mínima exigida pelo projeto. Este experimento reforça a capacidade da rede neural implementada de resolver tarefas reais de classificação utilizando apenas bibliotecas de baixo nível.
